In [22]:
import numpy as np
import pandas as pd
import xarray as xr
import glob

In [7]:
filedir_path = "../../data/raw/btsdelay/"
files = glob.glob(f"{filedir_path}*.parquet")

# df = pd.concat([pd.read_parquet(i) for i in files]).sample(1_000_000)
df = pd.read_parquet(files[0]).sample(1_000_000)


ERROR! Session/line number was not unique in database. History logging moved to new session 1776


In [9]:
df.describe()

,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,...,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,DistanceGroup,DivAirportLandings
count,1000000.000000,984997.000000,984761.000000,984761.000000,984059.000000,981935.000000,980809.000000,1000000.000000,982003.000000,1000000.000000,...,983389.000000,983391.000000,982863.000000,982861.000000,1000000.000000,981935.000000,981935.000000,981935.000000,1000000.000000,1000000.000000
mean,1330.179243,1334.395805,13.396219,10.078811,1462.873598,13.612724,110.088727,138.831684,134.361549,787.134580,...,17.037052,1357.815509,1458.414001,7.370528,1482.351853,5.343171,0.194550,-0.166419,3.623750,0.003635
std,491.178431,504.587379,43.881606,45.053659,540.201263,43.635164,70.568495,72.872598,72.578159,594.084553,...,9.790819,506.085019,536.299719,5.883326,521.075293,47.084781,0.395854,2.345179,2.341947,0.113172
min,1.000000,1.000000,0.000000,-611.000000,1.000000,0.000000,8.000000,-79.000000,4.000000,16.000000,...,0.000000,1.000000,1.000000,0.000000,1.000000,-475.000000,0.000000,-2.000000,1.000000,0.000000
25%,915.000000,917.000000,0.000000,-6.000000,1046.000000,0.000000,59.000000,86.000000,82.000000,349.000000,...,11.000000,932.000000,1042.000000,4.000000,1058.000000,-14.000000,0.000000,-1.000000,2.000000,0.000000
50%,1322.000000,1327.000000,0.000000,-2.000000,1503.000000,0.000000,92.000000,120.000000,116.000000,628.000000,...,14.000000,1341.000000,1500.000000,6.000000,1512.000000,-6.000000,0.000000,-1.000000,3.000000,0.000000
75%,1735.000000,1744.000000,7.000000,7.000000,1915.000000,8.000000,139.000000,169.000000,165.000000,1023.000000,...,20.000000,1800.000000,1910.000000,9.000000,1917.000000,8.000000,0.000000,0.000000,5.000000,0.000000
max,2359.000000,2400.000000,2625.000000,2625.000000,2400.000000,2635.000000,674.000000,1614.000000,728.000000,4983.000000,...,176.000000,2400.000000,2400.000000,220.000000,2400.000000,2635.000000,1.000000,12.000000,11.000000,9.000000


In [40]:
squared_delays = (df
                  .assign(sqrd_delay = lambda df_: df_['ArrDelay']**2)
                  [['Origin', 'Dest', 'sqrd_delay']]
                  .groupby(['Origin', 'Dest'])
                  .agg(mean_sqr_delay = pd.NamedAgg('sqrd_delay', np.sum),)
                  .reset_index()
                  )
squared_delays.head().values.tolist()

[['ABE', 'ATL', 390552.0],
 ['ABE', 'CLT', 140668.0],
 ['ABE', 'DTW', 329615.0],
 ['ABE', 'FLL', 72626.0],
 ['ABE', 'MYR', 448168.0]]

In [88]:

def construct_adj_mat(flights: pd.DataFrame) -> pd.DataFrame:
    """ajd matrix of form Aij => flight from i to j, id est rows are origin, cols are dest """

    squared_delays = (df
                      .assign(sqrd_delay = lambda df_: df_['ArrDelay']**2)
                      [['Origin', 'Dest', 'sqrd_delay']]
                      .groupby(['Origin', 'Dest'])
                      .agg(mean_sqr_delay = pd.NamedAgg('sqrd_delay', np.sum),)
                      .reset_index()
                      .values
                      .tolist()
                      )

    locs = np.unique(np.concatenate((flights['Origin'].unique(), flights['Dest'].unique())))
    

    adj_mat = xr.DataArray(pd.DataFrame(columns=locs, index=locs,
                                        data=np.zeros((len(locs), len(locs)))), 
                           dims=("Origin", "Dest"))
    
    for origin, dest, sqrd_delay in squared_delays:
        adj_mat.loc[{'Origin': origin, 'Dest': dest}] = sqrd_delay

    return adj_mat


adj_mat = construct_adj_mat(squared_delays)

In [92]:
adj_mat.mean()

<xarray.DataArray ()>
array(16106.34339664)

In [69]:
adj_mat.to_dataset(name='name')

<xarray.Dataset>
Dimensions:  (Origin: 370, Dest: 370)
Coordinates:
  * Origin   (Origin) object 'ABE' 'ABI' 'ABQ' 'ABR' ... 'YAK' 'YKM' 'YNG' 'YUM'
  * Dest     (Dest) object 'ABE' 'ABI' 'ABQ' 'ABR' ... 'YAK' 'YKM' 'YNG' 'YUM'
Data variables:
    name     (Origin, Dest) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0